In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12120)])
  except RuntimeError as e:
    print(e)

In [2]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# AUxillary methods
def getDist(y):
    ax = sns.countplot(y)
    ax.set(title="Count of data classes")
    plt.show()

def getData(dist, x, y):
    dx = []
    dy = []
    counts = [0 for i in range(10)]
    for i in range(len(x)):
        if counts[y[i]]<dist[y[i]]:
            dx.append(x[i])
            dy.append(y[i])
            counts[y[i]] += 1
        
    return np.array(dx), np.array(dy)

# Load and compile Keras model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

# Load dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
dist = [4000, 4000, 4000, 3000, 10, 10, 10, 10, 4000, 10]
x_train, y_train = getData(dist, x_train, y_train)
# getDist(y_train)

# Define Flower client
class FlowerClient(fl.client.NumPyClient):
    def get_parameters(self):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        r = model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), verbose=0)
        hist = r.history
        print("Fit history : " ,hist)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
        print("Eval accuracy : ", accuracy)
        return loss, len(x_test), {"accuracy": accuracy}

# Start Flower client
fl.client.start_numpy_client(
        server_address="localhost:8090", 
        client=FlowerClient(), 
        grpc_max_message_length = 1024*1024*1024
)


INFO flower 2022-07-11 11:33:32,910 | connection.py:102 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flower 2022-07-11 11:33:32,913 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flower 2022-07-11 11:33:32,914 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flower 2022-07-11 11:33:32,917 | connection.py:39 | ChannelConnectivity.READY


Fit history :  {'loss': [0.20463724434375763], 'accuracy': [0.9418372511863708], 'val_loss': [2.612410545349121], 'val_accuracy': [0.4977000057697296]}
Eval accuracy :  0.836899995803833
Fit history :  {'loss': [0.10669707506895065], 'accuracy': [0.969028890132904], 'val_loss': [1.8658994436264038], 'val_accuracy': [0.5741000175476074]}
Eval accuracy :  0.9136000275611877
Fit history :  {'loss': [0.07447303831577301], 'accuracy': [0.9781102538108826], 'val_loss': [1.4835240840911865], 'val_accuracy': [0.6654000282287598]}
Eval accuracy :  0.9308000206947327
Fit history :  {'loss': [0.05822255462408066], 'accuracy': [0.9815223217010498], 'val_loss': [1.6456648111343384], 'val_accuracy': [0.6593000292778015]}
Eval accuracy :  0.9351000189781189
Fit history :  {'loss': [0.0444846972823143], 'accuracy': [0.9857742786407471], 'val_loss': [1.431843638420105], 'val_accuracy': [0.6777999997138977]}
Eval accuracy :  0.9466000199317932
Fit history :  {'loss': [0.03756108507514], 'accuracy': [0.9

DEBUG flower 2022-07-11 11:34:25,074 | connection.py:121 | gRPC channel closed
INFO flower 2022-07-11 11:34:25,075 | app.py:101 | Disconnect and shut down


Eval accuracy :  0.9577999711036682
